In [0]:
# ======================================
# SILVER TRANSFORMATION - PRODUCTS
# ======================================
# Objetivo:
# Tomar datos crudos desde la capa Bronze (bronze_products)
# y generar una tabla Silver con:
# - Tipos de datos correctos
# - Precios normalizados
# - Duplicados eliminados con criterio
# - Datos listos para consumo analítico / dbt
# ======================================

from pyspark.sql.functions import (
    col,
    row_number
)

from pyspark.sql.window import Window



In [0]:
# -----------------------------
# Definición de catálogo y esquema
# -----------------------------

CATALOG = "workspace"
SCHEMA = "retail_medallion_pipeline"

BRONZE_TABLE = f"{CATALOG}.{SCHEMA}.bronze_products"
SILVER_TABLE = f"{CATALOG}.{SCHEMA}.silver_products"


In [0]:
# -----------------------------
# Lectura de datos Bronze
# -----------------------------

df_bronze = spark.table(BRONZE_TABLE)

# Inspección opcional
display(df_bronze)


In [0]:
# -----------------------------
# Transformaciones Silver
# -----------------------------

df_clean = (
    df_bronze
    # Conversión de tipos
    .withColumn("product_id", col("product_id").cast("int"))
    .withColumn("price", col("price").cast("double"))

    # Normalización de texto
    .withColumn("product_name", col("product_name"))
    .withColumn("category", col("category"))
    .withColumn("currency", col("currency"))
)


In [0]:
# -----------------------------
# Eliminación de duplicados
# -----------------------------
# Regla:
# Si existe más de un registro por product_id,
# conservamos uno arbitrario (no hay fecha de vigencia)

window_spec = (
    Window
    .partitionBy("product_id")
    .orderBy(col("product_id"))
)

df_silver = (
    df_clean
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)


In [0]:
# -----------------------------
# Validaciones de calidad
# -----------------------------

print("Total registros Silver:", df_silver.count())

print(
    "Products con product_id nulo:",
    df_silver.filter(col("product_id").isNull()).count()
)

print(
    "Products con precio negativo:",
    df_silver.filter(col("price") < 0).count()
)


In [0]:
# -----------------------------
# Escritura tabla Silver
# -----------------------------

(
    df_silver.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(SILVER_TABLE)
)


In [0]:
# -----------------------------
# Verificación final
# -----------------------------

spark.table(SILVER_TABLE).display()

